In [3]:
pip install -U transformers accelerate datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 19.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
  Attempting uninstall: transformers
    Found existing installation: transformers 5.0.0
    Uninstalling transformers-5.0.0:
      Successfully uninstalled transformers-5.0.0


In [1]:
import pandas as pd
import numpy as np
import torch
import random
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, BertConfig, BertForSequenceClassification
from datasets import Dataset


# 1. 환경 설정 (Seed 고정)
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 장치: {device}")

# 2. 데이터 로드 및 병합
# (1) DKTC 위협 데이터 로드
url = "https://raw.githubusercontent.com/tunib-ai/DKTC/main/data/train.csv"
try:
    df_threat = pd.read_csv(url)
    df_threat = df_threat[['class', 'conversation']]
    print(f"위협 데이터 로드: {len(df_threat)}개")
    # 확인용: 실제 클래스 이름 출력
    print(f"   └─ 클래스 목록: {df_threat['class'].unique()}")
except Exception as e:
    print(f"위협 데이터 로드 실패: {e}")

# (2) 일반 대화 데이터 로드
normal_file = "normal_conversation.csv"
try:
    df_normal = pd.read_csv(normal_file)
    # 화자 라벨 제거 전처리
    df_normal['conversation'] = df_normal['conversation'].str.replace(r'(^|\n)[AB]:\s*', '', regex=True)

    if 'class' not in df_normal.columns:
        df_normal['class'] = '일반 대화'

    df_normal = df_normal[['class', 'conversation']]
    print(f"일반 대화 데이터 로드: {len(df_normal)}개")

    # (3) 데이터 병합
    df_final = pd.concat([df_threat, df_normal], ignore_index=True)

    # 셔플
    df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)
    print(f"최종 데이터셋 병합 완료: {len(df_final)}개")

except FileNotFoundError:
    print(f"오류: '{normal_file}' 파일이 없습니다. 파일을 업로드했는지 확인해주세요.")
    df_final = df_threat.copy()

# 3. 라벨 인코딩 (수정됨: ' 대화' 추가)
label_map = {
    '협박 대화': 0,        # <-- 수정됨
    '갈취 대화': 1,        # <-- 수정됨
    '직장 내 괴롭힘 대화': 2, # <-- 수정됨
    '기타 괴롭힘 대화': 3,    # <-- 수정됨
    '협박': 0,            # (혹시 몰라 원본 키도 유지)
    '갈취': 1,
    '직장 내 괴롭힘': 2,
    '기타 괴롭힘': 3,
    '일반 대화': 4
}

df_final['label'] = df_final['class'].map(label_map)

# NaN 확인 (디버깅용)
if df_final['label'].isnull().sum() > 0:
    print(f"경고: 라벨링 실패한 데이터가 {df_final['label'].isnull().sum()}개 있습니다.")
    print(df_final[df_final['label'].isnull()]['class'].unique())
    # NaN 데이터 제거
    df_final = df_final.dropna(subset=['label'])
    df_final['label'] = df_final['label'].astype(int)

# 4. HuggingFace Dataset 변환
# Train / Validation 분리 (8:2)
train_df, val_df = train_test_split(df_final, test_size=0.2, random_state=42, stratify=df_final['label'])

# Dataset 객체 생성
train_dataset = Dataset.from_pandas(train_df[['conversation', 'label']])
val_dataset = Dataset.from_pandas(val_df[['conversation', 'label']])


# 5. 토크나이저 및 모델 준비
MODEL_NAME = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_function(examples):
    return tokenizer(examples["conversation"], truncation=True, max_length=256)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 모델 초기화(기존 Pretrained 삭제)

from transformers import BertConfig, BertForSequenceClassification

config = BertConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=768,
    num_hidden_layers=12,
    num_attention_heads=12,
    intermediate_size=3072,
    max_position_embeddings=512,
    num_labels=5
)

model = BertForSequenceClassification(config).to(device)

print("Pretrained를 삭제한 랜덤 초기화된 BERT 모델 생성")

# 6. 학습 (Training)
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": acc, "f1": f1}

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("\n학습 시작! (잠시만 기다려주세요...)")
trainer.train()

# 모델 저장
model.save_pretrained("./final_model")
tokenizer.save_pretrained("./final_model")
print("\n모델 저장 완료: ./final_model")

사용 장치: cuda
위협 데이터 로드: 3950개
   └─ 클래스 목록: ['협박 대화' '기타 괴롭힘 대화' '갈취 대화' '직장 내 괴롭힘 대화']
오류: 'normal_conversation.csv' 파일이 없습니다. 파일을 업로드했는지 확인해주세요.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

Step,Training Loss
500,1.122555


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]


모델 저장 완료: ./final_model


In [10]:
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import classification_report, accuracy_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 1. 설정 및 데이터 준비
MODEL_PATH = "./final_model"       # 학습된 모델 경로
df = df_final.copy()               # DATA_PATH 부분 삭제
df['conversation'] = df['conversation'].astype(str)

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 라벨 재매핑 (안전장치)
label_map = {
    '협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3,
    '협박': 0, '갈취': 1, '직장 내 괴롭힘': 2, '기타 괴롭힘': 3,
    '직장 괴롭힘': 2, '기타 괴롭힘': 3,
    '일반 대화': 4
}
df['label'] = df['class'].map(label_map)

# 결측치 제거
df = df.dropna(subset=['label'])
df['label'] = df['label'].astype(int)


# [핵심] 클래스별 100개씩 균형 샘플링
# 각 라벨별로 랜덤하게 100개씩 뽑습니다.
try:
    test_df = df.groupby('label').apply(lambda x: x.sample(n=100, random_state=42)).reset_index(drop=True)
    print(f"테스트 데이터셋 구성 완료: 총 {len(test_df)}개")
    print(test_df['class'].value_counts()) # 각 100개인지 확인
except ValueError as e:
    print(f"오류: 데이터가 부족하여 클래스별 100개를 뽑을 수 없습니다. ({e})")
    # 예외 시 전체 데이터 사용
    test_df = df

# 2. 모델 로드 및 예측
print("\n모델 로딩 중...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
    model.eval()
except OSError:
    print("오류: 저장된 모델을 찾을 수 없습니다. 학습 코드를 먼저 실행했는지 확인해주세요.")
    # (코드가 멈추지 않도록 임시 종료 처리 필요 시 exit())

# 예측 함수
def predict_batch(texts, batch_size=32):
    all_preds = []
    # 데이터가 많을 경우 배치를 나눠서 처리
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        all_preds.extend(preds)
    return np.array(all_preds)

print("예측 시작... (잠시만 기다려주세요)")
y_true = test_df['label'].tolist()
y_pred = predict_batch(test_df['conversation'].tolist())


# 3. 평가 지표 출력 (Text Only)
labels_present = sorted(set(y_true))

label_name_map = {
    0: '협박',
    1: '갈취',
    2: '직장 괴롭힘',
    3: '기타 괴롭힘',
    4: '일반 대화'
}

target_names = [label_name_map[i] for i in labels_present]

print(classification_report(
    y_true,
    y_pred,
    labels=labels_present,
    target_names=target_names,
    digits=4
))

print("\n" + "="*50)
print("[최종 모델 평가 리포트 (Test Set: 100 samples/class)]")
print("="*50)

# 1. Accuracy & F1-Score
acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted') # 각 클래스 비중이 같으므로 macro와 유사함

print(f"정확도 (Accuracy): {acc:.4f}")
print(f"F1 점수 (Weighted): {f1:.4f}")
print("-" * 50)

# 2. Classification Report (상세 지표)
print("\n[클래스별 상세 지표]")
# digits=4 옵션으로 소수점 4자리까지 정밀하게 출력
print(classification_report(y_true, y_pred, target_names=target_names, digits=4))
print("="*50)

/tmp/ipython-input-2950581055.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df = df.groupby('label').apply(lambda x: x.sample(n=100, random_state=42)).reset_index(drop=True)


테스트 데이터셋 구성 완료: 총 400개
class
협박 대화          100
갈취 대화          100
직장 내 괴롭힘 대화    100
기타 괴롭힘 대화      100
Name: count, dtype: int64

모델 로딩 중...


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

예측 시작... (잠시만 기다려주세요)
              precision    recall  f1-score   support

          협박     0.6204    0.6700    0.6442       100
          갈취     0.7582    0.6900    0.7225       100
      직장 괴롭힘     0.8774    0.9300    0.9029       100
      기타 괴롭힘     0.7158    0.6800    0.6974       100

    accuracy                         0.7425       400
   macro avg     0.7429    0.7425    0.7418       400
weighted avg     0.7429    0.7425    0.7418       400


[최종 모델 평가 리포트 (Test Set: 100 samples/class)]
정확도 (Accuracy): 0.7425
F1 점수 (Weighted): 0.7418
--------------------------------------------------

[클래스별 상세 지표]
              precision    recall  f1-score   support

          협박     0.6204    0.6700    0.6442       100
          갈취     0.7582    0.6900    0.7225       100
      직장 괴롭힘     0.8774    0.9300    0.9029       100
      기타 괴롭힘     0.7158    0.6800    0.6974       100

    accuracy                         0.7425       400
   macro avg     0.7429    0.7425    0.7418       400
weig

In [4]:
import pandas as pd
import json
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm


# 1. 설정 및 데이터 로드
MODEL_PATH = "./final_model"       # 학습된 모델 경로
TEST_FILE = "test.json"            # 업로드된 테스트 파일

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 장치: {device}")

# 테스트 데이터 로드 (JSON)
try:
    with open(TEST_FILE, 'r', encoding='utf-8') as f:
        test_data = json.load(f)
    print(f"테스트 데이터 로드 완료: {len(test_data)}개")

    # 데이터 구조 확인 (첫 번째 아이템)
    first_key = list(test_data.keys())[0]
    print(f"데이터 예시: ID={first_key}, 내용={test_data[first_key]}")

except FileNotFoundError:
    print("오류: 'test.json' 파일을 찾을 수 없습니다.")
    # (테스트용 더미 데이터)
    test_data = {"t_000": {"text": "이거 들어봐 와 이 노래 진짜 좋다"}, "t_001": {"text": "야 돈 내놔"}}


# 2. 모델 로드
print("모델 로딩 중...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
    model.eval()
except OSError:
    print("오류: 저장된 모델을 찾을 수 없습니다. 학습 코드를 먼저 실행했는지 확인해주세요.")
    # (임시 모델 로드)
    tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
    model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels=5).to(device)

# 3. 추론 (Inference)
print("예측 시작...")

results = []

# test_data가 딕셔너리 형태 {"t_000": {"text": "..."}} 라고 가정
for idx, item in tqdm(test_data.items()):
    # 텍스트 추출 (text 키가 없으면 conversation 등 다른 키 시도)
    text = item.get('text', item.get('conversation', ''))

    # 전처리 및 토크나이징
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # 예측값 (0~4)
    pred_label = torch.argmax(logits, dim=-1).item()

    # 결과 저장
    results.append({
        'idx': idx,
        'class': pred_label  # 숫자로 저장 (0, 1, 2, 3, 4)
    })

# 4. 제출 파일 생성
submission = pd.DataFrame(results)

# 컬럼 순서 정렬 (idx, class)
submission = submission[['idx', 'class']]

# 파일 저장
save_path = "submission.csv"
submission.to_csv(save_path, index=False)

print("\n" + "="*50)
print(f"제출 파일 생성 완료: {save_path}")
print("="*50)
print(submission.head())

사용 장치: cuda
오류: 'test.json' 파일을 찾을 수 없습니다.
모델 로딩 중...


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

예측 시작...


100%|██████████| 2/2 [00:00<00:00, 10.27it/s]



제출 파일 생성 완료: submission.csv
     idx  class
0  t_000      1
1  t_001      1
